Author: Omar El Malki (omar.elmalki@epfl.ch)

### Semantic Role Labling on first 10 resolved ROCStories

In [6]:
from typing import List
import re
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

from allennlp.common import JsonDict
# Semantic Role Labeling with BERT : https://github.com/Riccorl/transformer-srl
from transformer_srl import dataset_readers, models, predictors

In [7]:
def sentence_to_srl(sentence: str) -> JsonDict:
    """
    Extracts Semantic Roles from a sentence.

    :param sentence: sentence from which to extract semantic roles labels.
    :return: semantic_roles as PropBank English SRLs.
    """

    # Pre-trained model with BERT fine-tuned to predict PropBank SRLs on CoNLL 2012 dataset.
    predictor = predictors.SrlTransformersPredictor.from_path(
        "../data/pre-trained-transformer-srl/srl_bert_base_conll2012.tar.gz", "transformer_srl")

    # More documentation: https://docs.allennlp.org/models/main/models/structured_prediction/predictors/srl/
    semantic_roles: JsonDict = predictor.predict(sentence)
    return semantic_roles

In [8]:
def srl_to_triple(srl: JsonDict):
    """
    Extract Knowledge triples from semantic roles

    :param srl: PropBank English SRLs
    :return: knowledge triples as a List of Lists
    """
    res = []
    verbs = srl['verbs']
    n_empty = 0
    for d in verbs:
        tags = d['tags']
        triple = d['description']
        verb = d['verb']
        if 'B-ARG0' in tags:
            arg0 = re.search('\\[ARG0: (.*?)\\]', triple).group(1)
            args = re.findall('\\[ARG[1234M]+.*?: (.*?)\\]', triple)
            for arg1 in args:  
                res.append([arg0,verb,arg1])
        elif 'B-ARG1' in tags:
            arg1 = re.search('\\[ARG1: (.*?)\\]', triple).group(1)
            args = re.findall('\\[ARG[234M]+.*?: (.*?)\\]', triple)
            for arg2 in args:  
                res.append([arg1,verb,arg2])
        else:
            n_empty += 1
            print(f"Verb with no arguments: \n{d}\n")
    if not res:
        print(f"Empty triples for SRL: \n{srl}\n")
    return res

In [10]:
# Read ROCStories into pandas DataFrame
roc_stories_path_csv = '../generated/coreference_resolution/ROCStories_with_resolved_coreferences_first10.csv'
roc_stories_df = pd.read_csv(roc_stories_path_csv, sep='\t', header=0)

In [11]:
srl_df = roc_stories_df

In [12]:
# Add Knowledge triples to Dataframe for each sentence in the dataset
for n in range(1,6):
    srl_df[f'srl_r{n}'] = srl_df[f'resolved{n}'].progress_apply(lambda s: srl_to_triple(sentence_to_srl(s)))

 20%|██        | 2/10 [00:09<00:37,  4.69s/it]

Verb with no arguments: 
{'verb': 'had', 'description': 'David noticed David [have.01: had] put on a lot of weight recently .', 'tags': ['O', 'O', 'O', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'frame': 'have.01', 'frame_score': 1.0, 'lemma': 'have'}



 50%|█████     | 5/10 [00:34<00:38,  7.65s/it]

Verb with no arguments: 
{'verb': 'would', 'description': 'Bill thought a truck [go.04: would] be better for what Bill needed .', 'tags': ['O', 'O', 'O', 'O', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'frame': 'go.04', 'frame_score': 0.6008500456809998, 'lemma': 'would'}



 20%|██        | 2/10 [00:08<00:34,  4.31s/it]

...

 50%|█████     | 5/10 [00:35<00:38,  7.75s/it]

Verb with no arguments: 
{'verb': 'was', 'description': 'Bill [be.03: was] set in Bill ways with conventional thinking .', 'tags': ['O', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'frame': 'be.03', 'frame_score': 0.9999977350234985, 'lemma': 'be'}



 90%|█████████ | 9/10 [01:08<00:08,  8.45s/it]

Verb with no arguments: 
{'verb': 'could', 'description': "we decided we [have.02: could] [ARGM-NEG: n't] be apart so My first girlfriend moved in with me .", 'tags': ['O', 'O', 'O', 'B-V', 'B-ARGM-NEG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'frame': 'have.02', 'frame_score': 0.05789218470454216, 'lemma': 'could'}



100%|██████████| 10/10 [01:26<00:00,  8.62s/it]


In [13]:
srl_df

,Unnamed: 0,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5,coref_clusters,resolved1,resolved2,resolved3,resolved4,resolved5,srl_r1,srl_r2,srl_r3,srl_r4,srl_r5
0,0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,David Drops the Weight,David noticed he had put on a lot of weight recently.,He examined his habits to try and figure out the reason.,He realized he'd been eating too much fast food lately.,He stopped going to burger places and started a vegetarian diet.,"After a few weeks, he started to feel much better.","[David: [David, he, He, his, He, he, He, he]]",David noticed David had put on a lot of weight recently.,David examined David habits to try and figure out the reason.,David realized David'd been eating too much fast food lately.,David stopped going to burger places and started a vegetarian diet.,"After a few weeks, David started to feel much better.","[[David, noticed, David had put on a lot of weight recently], [David, put, a lot of weight], [David, put, recently]]","[[David, examined, David habits], [David, examined, to try and figure out the reason], [David, figure, the reason]]","[[David, realized, David'd been eating too much fast food lately], [David'd, eating, too much fast food], [David'd, eating, lately]]","[[David, stopped, going to burger places], [David, going, to burger places], [David, started, a vegetarian diet]]","[[to feel much better, started, After a few weeks], [David, feel, much better]]"
1,1,0beabab2-fb49-460e-a6e6-f35a202e3348,Frustration,Tom had a very short temper.,One day a guest made him very angry.,He punched a hole in the wall of his house.,Tom's guest became afraid and left quickly.,Tom sat on his couch filled with regret about his actions.,"[Tom: [Tom, him, He, his, Tom, Tom, his, his]]",Tom had a very short temper.,One day a guest made Tom very angry.,Tom punched a hole in the wall of Tom house.,Tom's guest became afraid and left quickly.,Tom sat on Tom couch filled with regret about Tom actions.,"[[Tom, had, a very short temper]]","[[a guest, made, One day], [a guest, made, Tom very angry]]","[[Tom, punched, a hole in the wall of Tom house]]","[[Tom 's guest, became, afraid], [Tom 's guest, left, quickly]]","[[Tom, sat, on Tom couch], [Tom, sat, filled with regret about Tom actions], [Tom, filled, with regret about Tom actions]]"
2,2,87da1a22-df0b-410c-b186-439700b70ba6,Marcus Buys Khakis,Marcus needed clothing for a business casual event.,All of his clothes were either too formal or too casual.,He decided to buy a pair of khakis.,The pair he bought fit him perfectly.,Marcus was happy to have the right clothes for the event.,"[Marcus: [Marcus, his, He, he, him, Marcus], a pair of khakis: [a pair of khakis, The pair he bought]]",Marcus needed clothing for a business casual event.,All of Marcus clothes were either too formal or too casual.,Marcus decided to buy a pair of khakis.,a pair of khakis fit Marcus perfectly.,Marcus was happy to have the right clothes for the event.,"[[Marcus, needed, clothing], [Marcus, needed, for a business casual event]]","[[All of Marcus clothes, were, either too formal or too casual]]","[[Marcus, decided, to buy a pair of khakis], [Marcus, buy, a pair of khakis]]",[],"[[Marcus, was, happy to have the right clothes for the event], [Marcus, have, the right clothes for the event]]"
3,3,2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9,Different Opinions,Bobby thought Bill should buy a trailer and haul it with his car.,Bill thought a truck would be better for what he needed.,Bobby pointed out two vehicles were much more expensive.,Bill was set in his ways with conventional thinking.,He ended up buying the truck he wanted despite Bobby's advice.,"[Bobby: [Bobby, Bobby, Bobby], Bill: [Bill, his, Bill, he, Bill, his, He, he], a trailer: [a trailer, it], a truck: [a truck, the truck he wanted]]",Bobby thought Bill should buy a trailer and haul a trailer with Bill car.,Bill thought a truck would be better for what Bill needed.,Bobby pointed out two vehicles were much 

In [15]:
# Convert Dataframe to csv
srl_df.to_csv('../generated/semantic-role-labeling/ROCStories_resolved_with_knowledge_triples_first10.csv', sep='\t')

### Only keep entities

In [20]:
def srl_to_triple_entities(srl: JsonDict):
    """
    Extract Knowledge triples from semantic roles

    :param srl: PropBank English SRLs
    :return: knowledge triples as a List of Lists
    """
    res = []
    verbs = srl['verbs']
    n_empty = 0
    for d in verbs:
        tags = d['tags']
        triple = d['description']
        verb = d['verb']
        if 'B-ARG0' in tags:
            arg0 = re.search('\\[ARG0: (.*?)\\]', triple).group(1)
            args = re.findall('\\[ARG[1234]+.*?: (.*?)\\]', triple)
            for arg1 in args:  
                res.append([arg0,verb,arg1])
        elif 'B-ARG1' in tags:
            arg1 = re.search('\\[ARG1: (.*?)\\]', triple).group(1)
            args = re.findall('\\[ARG[234]+.*?: (.*?)\\]', triple)
            for arg2 in args:  
                res.append([arg1,verb,arg2])
        else:
            n_empty += 1
            print(f"Verb with no arguments: \n{d}\n")
    if not res:
        print(f"Empty triples for SRL: \n{srl}\n")
    return res

In [21]:
# Read ROCStories into pandas DataFrame
roc_stories_path_csv = '../generated/coreference_resolution/ROCStories_with_resolved_coreferences_first10.csv'
roc_stories_df = pd.read_csv(roc_stories_path_csv, sep='\t', header=0)

In [22]:
srl_entities_df = roc_stories_df

In [23]:
# Add Knowledge triples to Dataframe for each sentence in the dataset
for n in range(1,6):
    srl_entities_df[f'srl_er{n}'] = srl_entities_df[f'resolved{n}'].progress_apply(lambda s: srl_to_triple_entities(sentence_to_srl(s)))

 20%|██        | 2/10 [00:08<00:35,  4.46s/it]

Verb with no arguments: 
{'verb': 'had', 'description': 'David noticed David [have.01: had] put on a lot of weight recently .', 'tags': ['O', 'O', 'O', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'frame': 'have.01', 'frame_score': 1.0, 'lemma': 'have'}



 50%|█████     | 5/10 [00:35<00:38,  7.80s/it]

Verb with no arguments: 
{'verb': 'would', 'description': 'Bill thought a truck [go.04: would] be better for what Bill needed .', 'tags': ['O', 'O', 'O', 'O', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'frame': 'go.04', 'frame_score': 0.6008500456809998, 'lemma': 'would'}



 90%|█████████ | 9/10 [01:10<00:08,  8.64s/it]

...

 80%|████████  | 8/10 [01:04<00:17,  8.89s/it]

Empty triples for SRL: 
{'verbs': [{'verb': 'finished', 'description': '[ARGM-TMP: About 5 minutes later] , [ARG0: I] [finish.01: finished] .', 'tags': ['B-ARGM-TMP', 'I-ARGM-TMP', 'I-ARGM-TMP', 'I-ARGM-TMP', 'O', 'B-ARG0', 'B-V', 'O'], 'frame': 'finish.01', 'frame_score': 0.9940926432609558, 'lemma': 'finish'}], 'words': ['About', '5', 'minutes', 'later', ',', 'I', 'finished', '.']}



 90%|█████████ | 9/10 [01:08<00:08,  8.41s/it]

Verb with no arguments: 
{'verb': 'could', 'description': "we decided we [have.02: could] [ARGM-NEG: n't] be apart so My first girlfriend moved in with me .", 'tags': ['O', 'O', 'O', 'B-V', 'B-ARGM-NEG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'frame': 'have.02', 'frame_score': 0.05789218470454216, 'lemma': 'could'}



100%|██████████| 10/10 [01:26<00:00,  8.62s/it]


In [25]:
srl_entities_df

,Unnamed: 0,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5,coref_clusters,resolved1,resolved2,resolved3,resolved4,resolved5,srl_er1,srl_er2,srl_er3,srl_er4,srl_er5
0,0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,David Drops the Weight,David noticed he had put on a lot of weight recently.,He examined his habits to try and figure out the reason.,He realized he'd been eating too much fast food lately.,He stopped going to burger places and started a vegetarian diet.,"After a few weeks, he started to feel much better.","[David: [David, he, He, his, He, he, He, he]]",David noticed David had put on a lot of weight recently.,David examined David habits to try and figure out the reason.,David realized David'd been eating too much fast food lately.,David stopped going to burger places and started a vegetarian diet.,"After a few weeks, David started to feel much better.","[[David, noticed, David had put on a lot of weight recently], [David, put, a lot of weight]]","[[David, examined, David habits], [David, figure, the reason]]","[[David, realized, David'd been eating too much fast food lately], [David'd, eating, too much fast food]]","[[David, stopped, going to burger places], [David, going, to burger places], [David, started, a vegetarian diet]]","[[David, feel, much better]]"
1,1,0beabab2-fb49-460e-a6e6-f35a202e3348,Frustration,Tom had a very short temper.,One day a guest made him very angry.,He punched a hole in the wall of his house.,Tom's guest became afraid and left quickly.,Tom sat on his couch filled with regret about his actions.,"[Tom: [Tom, him, He, his, Tom, Tom, his, his]]",Tom had a very short temper.,One day a guest made Tom very angry.,Tom punched a hole in the wall of Tom house.,Tom's guest became afraid and left quickly.,Tom sat on Tom couch filled with regret about Tom actions.,"[[Tom, had, a very short temper]]","[[a guest, made, Tom very angry]]","[[Tom, punched, a hole in the wall of Tom house]]","[[Tom 's guest, became, afraid]]","[[Tom, sat, on Tom couch], [Tom, filled, with regret about Tom actions]]"
2,2,87da1a22-df0b-410c-b186-439700b70ba6,Marcus Buys Khakis,Marcus needed clothing for a business casual event.,All of his clothes were either too formal or too casual.,He decided to buy a pair of khakis.,The pair he bought fit him perfectly.,Marcus was happy to have the right clothes for the event.,"[Marcus: [Marcus, his, He, he, him, Marcus], a pair of khakis: [a pair of khakis, The pair he bought]]",Marcus needed clothing for a business casual event.,All of Marcus clothes were either too formal or too casual.,Marcus decided to buy a pair of khakis.,a pair of khakis fit Marcus perfectly.,Marcus was happy to have the right clothes for the event.,"[[Marcus, needed, clothing]]","[[All of Marcus clothes, were, either too formal or too casual]]","[[Marcus, decided, to buy a pair of khakis], [Marcus, buy, a pair of khakis]]",[],"[[Marcus, was, happy to have the right clothes for the event], [Marcus, have, the right clothes for the event]]"
3,3,2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9,Different Opinions,Bobby thought Bill should buy a trailer and haul it with his car.,Bill thought a truck would be better for what he needed.,Bobby pointed out two vehicles were much more expensive.,Bill was set in his ways with conventional thinking.,He ended up buying the truck he wanted despite Bobby's advice.,"[Bobby: [Bobby, Bobby, Bobby], Bill: [Bill, his, Bill, he, Bill, his, He, he], a trailer: [a trailer, it], a truck: [a truck, the truck he wanted]]",Bobby thought Bill should buy a trailer and haul a trailer with Bill car.,Bill thought a truck would be better for what Bill needed.,Bobby pointed out two vehicles were much more expensive.,Bill was set in Bill ways with conventional thinking.,Bill ended up buying a truck despite Bobby's advice.,"[[Bobby, thought, Bill should buy a trailer and haul a trailer with Bill car], [Bill, should, buy a trailer], [Bill, buy, a trailer], [Bill, haul, a trailer]]","[[Bill, thought, a 

In [26]:
# Convert Dataframe to csv
srl_entities_df.to_csv('../generated/semantic-role-labeling/ROCStories_resolved_with_knowledge_triples_entities_only_first10.csv', sep='\t')